In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
import torch
import re

In [3]:
cos = torch.nn.CosineSimilarity(dim=0, eps=1e-6)

In [4]:
def find_k_nearest_neighbors(word, embedding_dict, k):
  words_cosine_similarity = dict()
  for token in embedding_dict.keys():
    words_cosine_similarity[token] = cos(embedding_dict[word], embedding_dict[token]).item()
  words_cosine_similarity = dict(sorted(words_cosine_similarity.items(), key=lambda item: item[1]))
  return list(words_cosine_similarity.keys())[-k:][::-1]

def delete_hashtag_usernames(text):
  try:
    result = []
    for word in text.split():
      if word[0] not in ['@', '#']:
        result.append(word)
    return ' '.join(result)
  except:
    return ''

def delete_url(text):
  text = re.sub(r'http\S+', '', text)
  return text

In [5]:
word = 'زندگی'
k = 10

#0. Data preprocessing

In [6]:
!pip install json-lines

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
import json_lines
import pandas as pd

In [9]:
# 1. extract all tweets from files and save them in memory base on each year.
!unzip "gdrive/MyDrive/data/mahsa_amini_data.zip"
# 2. remove urls, hashtags and usernames.
text = []
text = pd.read_csv("mahsa_amini_data.csv")['Text']

Archive:  gdrive/MyDrive/data/mahsa_amini_data.zip
replace mahsa_amini_data.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [10]:
text

0        بنشین تا شود نقش فال ما \nنقش هم‌ فردا شدن\n#م...
1        @Tanasoli_Return @dr_moosavi این گوزو رو کی گر...
2        @ghazaleghaffary برای ایران، برای مهسا.\n#OpIr...
3        @_hidden_ocean مرگ بر دیکتاتور \n#OpIran \n#Ma...
4        نذاریم خونشون پایمال شه.‌‌.‌‌.\n#Mahsa_Amini #...
                               ...                        
19995    برای ایران بانو #Mahsa_Amini      #MahsaAmini ...
19996    @MohammadTehra16 @mimpedram از بس حاج خانم درا...
19997    به افتخار از بین رفتن جمهوری اسلامی🙆‍♂️🙆‍♂️🙆‍♂...
19998    پنجاه و شیش \n\n#مهسا_امینی \n#Mahsa_Amini \n#...
19999    در محیط طوفان‌زای ماهرانه در جنگ است\nناخدای ا...
Name: Text, Length: 20000, dtype: object

In [11]:
new_text = []
for t in text:
  deleted_hashtags_usernames = delete_hashtag_usernames(t)
  new_text.append(delete_url(deleted_hashtags_usernames))
new_text

['بنشین تا شود نقش فال ما نقش هم\u200c فردا شدن',
 'این گوزو رو کی گردن میگیره؟؟ دچار زوال عقل شده از بس پای منبر دستمال کشی کرده.',
 'برای ایران، برای مهسا.',
 'مرگ بر دیکتاتور',
 'نذاریم خونشون پایمال شه.\u200c\u200c.\u200c\u200c.',
 'مابهت افتخار میکنیم نبات باعث شدی کل دنیا مارو ببینه',
 'برای انسانای خوشگلمون',
 'فارغ از هر باوری متحد شویم.',
 'اینها عجب موجودات پستی هستن🥺🥺🥺الهی بگردم، من خودم باردارم و حتی توتظاهرات مسالمت امیز خارج ایران استرس داشتم ادم ها نا خود اگاه بهم ضربه بزنن،بمیرم برای دل اون زن که چه کشیده...مرگ بر دیکتاتور',
 'کصخلا چرا ۴ تاوفحشش نمیدن؟',
 'و از روی پول ملی....',
 'برای آزادی ایران تا آخرین قطره خونم میجنگم 🇮🇷✌🏻 (7) ',
 'داریم چهل تا میشیم',
 'سی و سه',
 'سلام بچه ها اگر من رو میبینید حتما هشتگ بزنید.',
 'امروز برای کاری رفتم سراغ وسایل قدیمی و اینارو دیدم : برای تمام روزهایی که رفتم دکه تا کارت دیال آپ بخرم و یه چیزایی تو وبلاگم بنویسم و 88 زدید آرشیوم و حذف کردید برای اون روزها خیلی بغض دارم. ',
 'فالو کنید بک بدم',
 'زن،زندگی،آزادی.',
 'دستاشو مشت کر

#1. One hot encoding

In [12]:
from collections import Counter
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import tensorflow as tf
from keras.utils import to_categorical

In [13]:
# 1. find one hot encoding of each word for each year
# 2. find 10 nearest words from "ولنتاین"
unique_words = set()
for text in new_text:
  for t in text.split():
    unique_words.add(t)
#print(len(unique_words))

one_hot_encoded = []
embedded_words = {}
for i, word in enumerate(unique_words):
  temp = np.zeros(len(unique_words))
  temp[i] = 1
  one_hot_encoded.append(temp)
  embedded_words[word] = torch.from_numpy(one_hot_encoded[i])

find_k_nearest_neighbors(word, embedded_words, k)

['نبینیمش',
 'بدهند…',
 'همهههه',
 'زمین...',
 'نتونه',
 'نگیریم!💪💪💪',
 'فالوورز',
 'این\u200cچند\u200cوقت\u200cدولت',
 'یدونه',
 '۶']

#Advantages

*   It’s relatively straightforward to implement and enables us to apply machine-learning algorithms to data with categorical columns.


#Disadvantages

*   It increases dimensionality so training becomes slower and more complex.
*   It can also create sparse data since most entries in the new columns will be zero.
*   It takes more space but adds no new information since it only changes data representation.


#2. TF-IDF

In [29]:
tweet = new_text[10]
print(tweet)

و از روی پول ملی....


In [22]:
words_dict = {}
tf_idf_dict = {}

for i, word in enumerate(unique_words):
  for token in new_text:
    if word in token:
      try:
        words_dict[word] += 1
      except:
        words_dict[word] = 1

In [26]:
print(words_dict)

defaultdict(<class 'int'>, {'حسودیم': 3, 'راکیستمون✌🏻': 1, 'میبندن': 1, 'ما✌️': 1, 'بازم\u200cکمه': 1, 'میرساند': 2, 'آزادی....': 7, 'کیدز': 1, 'جد': 211, 'ضعیفه.': 1, '(هفتاد': 1, 'Sharia': 3, '،تو': 10, 'ساتور': 1, 'جسمي': 1, 'نمایندگان': 5, 'فروختید': 3, 'بماند': 6, 'بترسد🚫': 1, 'خبرررررر؟': 1, 'سیوپن': 1, 'جام\u200cجم': 1, 'غواصان': 1, 'مرسی❤️': 1, 'درستهههههه!': 1, 'پیامهای': 2, 'شاهدی': 1, 'پلیسا': 12, 'ادیپ': 1, 'استارباکس': 1, 'مامانت': 5, 'بینهم»': 1, 'نقطه': 13, '۴۴سال': 1, 'نوشهر': 3, 'قزوین': 8, 'گرنده': 1, 'زیبایی': 29, 'لحظه\u200cای': 3, '?!': 2, 'معتقد': 4, 'رانندگان': 1, '✌🌱❤': 1, 'موشک': 6, 'درسته؟درسته': 1, 'جنده': 44, '"تکرار': 1, 'تقلب\u200cها': 1, 'روزايي': 2, 'هرکسی،': 1, 'میزدی؟!\U0001f972': 1, 'غصب': 2, 'آرامش': 27, 'خَدَم': 1, 'نمیبینه،': 1, 'بخون...': 1, 'گهی': 11, 'طبقه': 4, 'روزنامەنگاران': 1, 'شش.': 3, 'سوزوندین': 2, 'زایمانو': 1, 'و\u200cناموسشون': 1, 'نشینا': 4, 'مستقيم': 1, 'کابوس؟': 1, 'طبیعتن': 1, 'می\u200cشنوید!!': 1, 'نگیرین،': 1, 'عزم': 1, 'تعفنش': 

In [30]:
for i, text in enumerate(new_text):
  tokens = text.split(' ')
  tf_idf = np.zeros(len(new_text))
  for j, token in enumerate(tokens):
    if(words_dict[token] != 0):
      tf_idf[j] = float(tokens.count(token)/len(tokens)) * (np.log(float(len(new_text)/words_dict[token])))
  tf_idf_dict[text] = torch.tensor(tf_idf)

find_k_nearest_neighbors(tweet, tf_idf_dict, k)

['و از روی پول ملی....',
 'از او بالا ساندیس میایَه...',
 'ما همه باهم هستیم ✊🏻❤️ ',
 'ی روز خوب نمیاد میسازیمش',
 'برای هر روز استرس ایندمون',
 'با مردم باشید فردا دیره',
 'ما هم داریم هشتگ میشیم😞',
 'به امید ۴۰ میلیون منشن.',
 'یک دو سه چار پنچ',
 'برای همه نکنه کسی ببینتمونا']

#Advantages
[Auxiliary link](https://www.quora.com/What-are-the-advantages-and-disadvantages-of-TF-IDF) 

*  Easy to compute.
*  You have some basic metric to extract the most descriptive terms in a document.
*  You can easily compute the similarity between 2 documents using it.

#Disadvantages
[Auxiliary link](https://www.quora.com/What-are-the-advantages-and-disadvantages-of-TF-IDF) 

*  TF-IDF is based on the bag-of-words (BoW) model, therefore it does not capture position in text, semantics, co-occurrences in different documents, etc.
*  For this reason, TF-IDF is only useful as a lexical level feature.
*  Cannot capture semantics (e.g. as compared to topic models, word embeddings).

#3. Word2Vec

In [45]:
from gensim.models import Word2Vec
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [57]:
data = [] 
for text in new_text:
  tokens = list(text.split(' '))
  data.append(tokens)

print(data)

[['بنشین', 'تا', 'شود', 'نقش', 'فال', 'ما', 'نقش', 'هم\u200c', 'فردا', 'شدن'], ['این', 'گوزو', 'رو', 'کی', 'گردن', 'میگیره؟؟', 'دچار', 'زوال', 'عقل', 'شده', 'از', 'بس', 'پای', 'منبر', 'دستمال', 'کشی', 'کرده.'], ['برای', 'ایران،', 'برای', 'مهسا.'], ['مرگ', 'بر', 'دیکتاتور'], ['نذاریم', 'خونشون', 'پایمال', 'شه.\u200c\u200c.\u200c\u200c.'], ['مابهت', 'افتخار', 'میکنیم', 'نبات', 'باعث', 'شدی', 'کل', 'دنیا', 'مارو', 'ببینه'], ['برای', 'انسانای', 'خوشگلمون'], ['فارغ', 'از', 'هر', 'باوری', 'متحد', 'شویم.'], ['اینها', 'عجب', 'موجودات', 'پستی', 'هستن🥺🥺🥺الهی', 'بگردم،', 'من', 'خودم', 'باردارم', 'و', 'حتی', 'توتظاهرات', 'مسالمت', 'امیز', 'خارج', 'ایران', 'استرس', 'داشتم', 'ادم', 'ها', 'نا', 'خود', 'اگاه', 'بهم', 'ضربه', 'بزنن،بمیرم', 'برای', 'دل', 'اون', 'زن', 'که', 'چه', 'کشیده...مرگ', 'بر', 'دیکتاتور'], ['کصخلا', 'چرا', '۴', 'تاوفحشش', 'نمیدن؟'], ['و', 'از', 'روی', 'پول', 'ملی....'], ['برای', 'آزادی', 'ایران', 'تا', 'آخرین', 'قطره', 'خونم', 'میجنگم', '🇮🇷✌🏻', '(7)', ''], ['داریم', 'چهل', 'تا', '

In [62]:
# 1. train a word2vec model base on all tweets for each year.
# 2. find 10 nearest words from "ولنتاین"

model_word2vec = Word2Vec(data, min_count = 1, window = 5)

words_dict={}
for word in unique_words:
  words_dict[word]=torch.tensor(model_word2vec[word])

find_k_nearest_neighbors(word, words_dict, k)

<ipython-input-62-56b45af7914d>:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  words_dict[word]=torch.tensor(model_word2vec[word])


['بریزین',
 'ميگه؟',
 'نمون',
 'نوشت',
 'اورە.',
 'ارتجاع',
 'میخواد!',
 'دشمنه',
 '🙂',
 'زندگیم']

#Advantages
[Auxiliary link](https://www.quora.com/What-are-the-advantages-and-disadvantages-of-Word2vec-and-GloVe) 

*  The data can be fed into the model in an online way and needs little preprocessing, thus requires little memory.
*  It is simple for a freshman to understand the principle and do implementation.
*  The mapping between the target word to its context word implicitly embeds the sub-linear relationship into the vector space of words, so that relationships like “king:man as queen:woman” can be infered by word vectors.
*  The idea is very intuitive, which transforms the unlabled raw corpus into labeled data (by mapping the target word to its context word), and learns the representation of words in a classification task.

#Disadvantages
[Auxiliary link](https://www.quora.com/What-are-the-advantages-and-disadvantages-of-Word2vec-and-GloVe) 

*  The sub-linear relationships are not explicitly defined. There is little theoretical support behind such characteristic.
*  The model could be very difficult to train if use the softmax function, since the number of categories is too large (the size of vocabulary). Though approxination algorithms like negative sampling (NEG) and hierarchical softmax (HS) are proposed to address the issue, other problems happen. For example, the word vectors by NEG are not distributed uniformally, they are located within a cone in the vector space hence the vector space is not sufficiently utilized.

#4. Contextualized embedding

In [ ]:
model_checkpoint = "HooshvareLab/bert-base-parsbert-uncased"

In [ ]:
# 1. fine tune a bert model base on all tweets for each year.
# 2. find 10 nearest words from "ولنتاین"

#Advantages
[Axiliary link](https://www.revuze.it/blog/bert-nlp/)
*  It’s bidirectional.
*  It combines Mask Language Model (MLM) and Next Sentence Prediction (NSP).
*  It’s the best method in NLP to understand context-heavy texts

# Disadvanteges


*   The model is large because of the training structure and corpus.
*   It is slow to train because it is big and there are a lot of weights to update.
*   It is expensive.

